In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import os
import random
import tensorflow as tf
import cv2 as cv

In [2]:
# Get images
image = img_to_array(load_img('images/waterfall.jpg'))
image = cv.resize(image, (400, 400))
image = np.array(image, dtype=float)

In [3]:
X = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,0]
Y = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [4]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [5]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [6]:
# Train model
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=500)

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 0.3390
Epoch 2/500
1/1 [==============================] - 1s 612ms/step - loss: 0.9466
Epoch 3/500
1/1 [==============================] - 1s 553ms/step - loss: 0.3213
Epoch 4/500
1/1 [==============================] - 1s 551ms/step - loss: 0.0137
Epoch 5/500
1/1 [==============================] - 1s 552ms/step - loss: 0.0252
Epoch 6/500
1/1 [==============================] - 1s 598ms/step - loss: 0.0641
Epoch 7/500
1/1 [==============================] - 1s 550ms/step - loss: 0.0269
Epoch 8/500
1/1 [==============================] - 1s 569ms/step - loss: 0.0078
Epoch 9/500
1/1 [==============================] - 1s 564ms/step - loss: 0.0074
Epoch 10/500
1/1 [==============================] - 1s 567ms/step - loss: 0.0070
Epoch 11/500
1/1 [==============================] - 1s 611ms/step - loss: 0.0068
Epoch 12/500
1/1 [==============================] - 1s 659ms/step - loss: 0.0066
Epoch 13/500
1/1 [======================

In [7]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output = output * 128


1/1 [==============================] - 2s 2s/step - loss: 5.4755e-04
0.000547545263543725
1/1 [==============================] - 2s 2s/step


In [8]:
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

cv_result = cv.cvtColor(np.float32(cur),  cv.COLOR_LAB2BGR)
cv_result = cv_result * 255

cv_result = cv_result.round(0)

print(cv_result[0][0])

cv.imwrite("img_result_cv.png", cv_result)

[230. 226. 192.]


True

In [9]:
model.save('model.h5')

In [10]:
from keras.models import load_model

model = load_model('model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 8)     80        
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 8)     584       
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 16)    1168      
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 16)    2320      
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 32)    4640      
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 up_sampling2d (UpSampling2D  (None, None, None, 32)   0